In [1]:
import numpy as np

train_features = np.load("../MCM/features/ImageNet_image_features.npz")['image_features']

text_features = np.load("../MCM/features/test_labels.npy")

np_data = np.load("../MCM/features/ImageNet_image_features.npz")
id_features = np_data['image_features']
id_labels = np_data['labels']

ood_features = []
for ood_dataset in [
    "iNaturalist", 
    "places365",
    "SUN", 
    "dtd"
]:
    np_data = np.load(f"../MCM/features/{ood_dataset}_image_features.npz")
    ood_features.append(np_data['image_features'])
ood_features = np.concatenate(ood_features, axis=0)

In [2]:
train_sim = train_features @ text_features.T
train_scores = np.max(train_sim, axis=1)
train_assigned = np.argmax(train_sim, axis=1)

id_sim = id_features @ text_features.T
id_scores = np.max(id_sim, axis=1)
id_assigned = np.argmax(id_sim, axis=1)

ood_sim = ood_features @ text_features.T
ood_scores = np.max(ood_sim, axis=1)
ood_assigned = np.argmax(ood_sim, axis=1)

In [8]:
# Class-wise thresholds
avg_thres = np.percentile(train_scores, 5)
print(avg_thres)

cls_thres = {}

for i in range(1000):
    cls_train_scores = train_scores[train_assigned==i]
    if False: # len(cls_train_scores) > 10:
        cls_thres[i] = np.percentile(cls_train_scores, 5)
    else:
        cls_thres[i] = avg_thres
cls_thres

0.3184665635228157


{0: 0.3184665635228157,
 1: 0.3184665635228157,
 2: 0.3184665635228157,
 3: 0.3184665635228157,
 4: 0.3184665635228157,
 5: 0.3184665635228157,
 6: 0.3184665635228157,
 7: 0.3184665635228157,
 8: 0.3184665635228157,
 9: 0.3184665635228157,
 10: 0.3184665635228157,
 11: 0.3184665635228157,
 12: 0.3184665635228157,
 13: 0.3184665635228157,
 14: 0.3184665635228157,
 15: 0.3184665635228157,
 16: 0.3184665635228157,
 17: 0.3184665635228157,
 18: 0.3184665635228157,
 19: 0.3184665635228157,
 20: 0.3184665635228157,
 21: 0.3184665635228157,
 22: 0.3184665635228157,
 23: 0.3184665635228157,
 24: 0.3184665635228157,
 25: 0.3184665635228157,
 26: 0.3184665635228157,
 27: 0.3184665635228157,
 28: 0.3184665635228157,
 29: 0.3184665635228157,
 30: 0.3184665635228157,
 31: 0.3184665635228157,
 32: 0.3184665635228157,
 33: 0.3184665635228157,
 34: 0.3184665635228157,
 35: 0.3184665635228157,
 36: 0.3184665635228157,
 37: 0.3184665635228157,
 38: 0.3184665635228157,
 39: 0.3184665635228157,
 40: 0.318

In [9]:
# id_res = []
ood_res = []

for i in range(1000):
    # cls_id_scores = id_scores[id_assigned == i]
    cls_ood_scores = ood_scores[ood_assigned == i]

    # id_res.append((cls_id_scores > cls_thres[i]).astype(int))
    ood_res.append((cls_ood_scores > cls_thres[i]).astype(int))

# id_res = np.concatenate(id_res)
ood_res = np.concatenate(ood_res)
ood_res[:5]

array([0, 0, 0, 0, 0])

In [10]:
# FPR
np.round(np.sum(ood_res) / len(ood_res) * 100, 2)

0.19